## Introduction

In this notebook, we will try to obtain the unique headers amongst all the csv files that we are going to merge. Then we will finally make a master comma seperated value file with list of headers that we obtain from this notebook

## Data Cleaning

### Import the necessary libaries 

In [ ]:
import pandas as pd
import numpy as np
import glob
import csv
import re
import os

### Setting up parent directory and sub directory

In [ ]:
parent_dir = "../UnifiedCSVTestData/"
filelist = []
dirs = []

def makefilelist(parent_dir):
    headers = []
    csv_headers = []
    subject_dirs = [os.path.join(parent_dir, dir) for dir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, dir))]
    filelist = []
    for dir in subject_dirs:
        csv_files = [os.path.join(dir, csv) for csv in os.listdir(dir) if os.path.isfile(os.path.join(dir, csv)) and csv.endswith('.csv')]
        for file in csv_files:
            filelist.append(file)
    
    return filelist, subject_dirs

### Read headers from CSV files

In [ ]:
def readCSV(fileList, subject_dirs):
    master_csv_headers = []
    for filename in fileList:
        slash = filename.split("/")
        parts = slash[2]
        subparts = parts.split("_")
        CIK = subparts[0]
        report_type = subparts[1]
        subsubpart = subparts[2].split('-')
        report_year = subsubpart[1]    
        df = pd.read_csv(filename,engine='python')
        df['CIK'] = CIK
        df['Reporting Type'] = report_type
        df['Report Year'] = report_year
        df.to_csv(filename, encoding='utf-8', index=False)
        
        with open(filename, 'r') as f:
            d_reader = csv.DictReader(f)
            headers = d_reader.fieldnames
            for header in headers:
                master_csv_headers.append(header)
            
    return master_csv_headers

In [ ]:
filelist, dirs = makefilelist(parent_dir)
csv_headers = readCSV(filelist, dirs)

Now we will only use the unqiue headers that we might require for our master CSV

In [ ]:
def uniqueHeaders(csv_headers):
    return set(csv_headers)

def chomp(list1):
    list1 = [x.replace('\n', '') for x in list1]
    return list1

Checking our unique header values and taking a look on the kind of values we have

In [304]:
final_headers = uniqueHeaders(csv_headers)
print(len(uniqueHeaders(csv_headers)))
lol = chomp(final_headers)
print(len(lol))

1837
1837


### Checking for different instances of similar headers

Using this to test out the best conditions required to extract all the possible variants of each header in the unified CSV

In [307]:
counter = 0
for header in lol:
    if 'description' in lol:
        print(header)
        counter+=1
print ("Counter", counter)

Counter 0


We will usse the above function as a way to guage how many similar ways are there to report a specific header and then merge all the occurances together to the master csv when we are conducting the joining

### Normalizing the values present in the column

In [ ]:
def normalizeValues(filelist):
    for filename in filelist:
        with open(filename, 'r') as f:
            df = pd.read_csv(filename, engine='python', dtype=str)
            headers = list(df)
            for header in headers:
                if '000' in header:
                    df[header] = df[header].astype(str) + '000'
            print (df)
            print("Filename", filename)        
            df.to_csv(filename, index=False, header=True)

In [ ]:
normalizeValues(filelist)

### Generate conditions for each type of Header 

In this section, we will decide the final size of the unified CSV that we are going to generate for each time user searches for something. 

In [300]:
column_headers = ["CIK", "Reporting Type", "Reporting Year", "Counterparty", "Notional Amount", "Reference Entity/Obligation", "Fixed Rate", "Expiration Date", "Appreciation/Depreciation", "Upfront Payments Paid/Received", "Implied Credit Spread", "Buy/Sell Protection", "Description"]
unified_csv = pd.DataFrame()
CIK_csv = pd.DataFrame(columns=["CIK"])
Reporting_Type_csv = pd.DataFrame(columns=["Reporting Type"])
Reporting_Year_csv = pd.DataFrame(columns=["Reporting Year"])
Counterparty_csv = pd.DataFrame(columns=["Counterparty"])
Notional_Amount_csv = pd.DataFrame(columns=["Notional Amount"])
Reference_Entity_csv = pd.DataFrame(columns=["Reference Entity/Obligation"])
Fixed_Rate_csv = pd.DataFrame(columns=["Fixed Rate"])
Expiration_Date_csv = pd.DataFrame(columns=["Expiration Date"])
Appreciation_csv = pd.DataFrame(columns=["Appreciation/Depreciation"])
Upfront_Payments_csv = pd.DataFrame(columns=["Upfront Payments Paid/Received"])
Buy_Sell_csv = pd.DataFrame(columns=["Buy/Sell Protection"])
Description_csv = pd.DataFrame(columns=["Description"]) 

#### Joining the headers and merging them to a final CSV

In [301]:
for filename in filelist:
    df = pd.read_csv(filename, engine='python', dtype=str)
    df = df.replace(r'\n',' ', regex=True)
    df = df.replace(r'\t',' ', regex=True) 
    headers = list(df)
    for header in headers:
        #CIK NUMBER
        if 'cik' in header.lower():
            for i in df[header].iteritems():
                CIK_csv = CIK_csv.append({'CIK': i[1]}, ignore_index=True)
            else:
            CIK_csv = CIK_csv.append({'CIK': "NaN"}, ignore_index=True)
        
        if 'reporting' in header.lower():
            for i in df[header].iteritems():
                Reporting_Type_csv = Reporting_Type_csv.append({'Reporting Type': i[1]}, ignore_index=True)
        else:
            Reporting_Type_csv = Reporting_Type_csv.append({'Reporting Type': "NaN"}, ignore_index=True)
        
        if 'year' in header.lower():
            for i in df[header].iteritems():
                Reporting_Year_csv = Reporting_Year_csv.append({'Reporting Year': i[1]}, ignore_index=True)
        
        if 'counter' in header.lower() or 'party' in header.lower() or 'obligation' in header.lower():
            for i in df[header].iteritems():
                Counterparty_csv = Counterparty_csv.append({'Counterparty': i[1]}, ignore_index=True)
        
                
        if 'notional' in header.lower() or 'amount' in header.lower() or 'value' in header.lower():
            for i in df[header].iteritems():
                Notional_Amount_csv = Notional_Amount_csv.append({'Notional Amount': i[1]}, ignore_index=True)
                               
        if 'reference' in header.lower() or ('entity' in header.lower() or 'obligation' in header.lower()):
            for i in df[header].iteritems():
                Reference_Entity_csv = Reference_Entity_csv.append({'Reference Entity/Obligation': i[1]}, ignore_index=True)
        
        if 'fixed' in header.lower() or 'rate' in header.lower():
            for i in df[header].iteritems():
                Reference_Entity_csv = Reference_Entity_csv.append({'Fixed Rate': i[1]}, ignore_index=True)
                
        if 'date' in header.lower() or 'expiration' in header.lower() or 'termination' in header.lower() or 'maturity' in header.lower():
            for i in df[header].iteritems():
                Expiration_Date_csv = Expiration_Date_csv.append({'Expiration Date': i[1]}, ignore_index=True)
        
        if 'appreciation' in header.lower() or 'depriciation' in header.lower():
            for i in df[header].iteritems():
                Appreciation_csv = Appreciation_csv.append({'Appreciation/Depreciation': i[1]}, ignore_index=True)
        
        if 'upfront' in header.lower() or  ('payment' in header.lower() or 'premium' in header.lower()):
            for i in df[header].iteritems():
                Upfront_Payments_csv = Upfront_Payments_csv.append({'Upfront Payments Paid/Received': i[1]}, ignore_index=True)
        
        if 'buy' in header.lower() or  'sell' in header.lower() or 'protection' in header.lower():
            for i in df[header].iteritems():
                Buy_Sell_csv = Buy_Sell_csv.append({'Buy/Sell Protection': i[1]}, ignore_index=True)
                
        if 'description' in header.lower():
            for i in df[header].iteritems():
                Description_csv = Description_csv.append({'Description': i[1]}, ignore_index=True)
        else:
            Description_csv = Description_csv.append({'Description': "NaN"}, ignore_index=True)
            
    

Let's check the final result

In [302]:
Description_csv

,Description
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [303]:
unified_csv = CIK_csv.join(Reporting_Type_csv)
unified_csv = unified_csv.join(Reporting_Year_csv)
unified_csv = unified_csv.join(Counterparty_csv)
unified_csv = unified_csv.join(Notional_Amount_csv)
unified_csv = unified_csv.join(Reference_Entity_csv)
unified_csv = unified_csv.join(Expiration_Date_csv)
unified_csv = unified_csv.join(Appreciation_csv)
unified_csv = unified_csv.join(Upfront_Payments_csv)
unified_csv = unified_csv.join(Buy_Sell_csv)
unified_csv = unified_csv.join(Description_csv)
unified_csv

,CIK,Reporting Type,Reporting Year,Counterparty,Notional Amount,Reference Entity/Obligation,Fixed Rate,Expiration Date,Appreciation/Depreciation,Upfront Payments Paid/Received,Buy/Sell Protection,Description
0,NaN,NaN,07,Deutsche Bank AG,2110000,NaN,1.0,06/20/16,"$152,115","1,679,690",Buy,NaN
1,NaN,NaN,10,Credit Suisse International,1550000,NaN,5.0,03/20/11,"(134,444",$258000,Buy,NaN
2,NaN,NaN,10,Credit Suisse International,2150000,NaN,3.3,03/20/11,"(1,185,433",111000,Buy,NaN
3,NaN,NaN,10,Credit Suisse International,4000000,NaN,1.0,05/20/12,"(416,466",$195.00000,Buy,NaN
4,0000880943,NaN,10,JPMorgan Chase Bank,10000000,"CDX Emerging Markets Index Series 25, Version 1",NaN,03/20/15,$(283000,$195000,Buy,NaN
5,NaN,N-CSRS,16,Barclays Bank PLC,"25,000,000",NaN,(1.000,06/20/21,(441000,$195000,Buy,NaN
6,NaN,NaN,16,"CDX Emerging Markets Index Series 25, Version 1","1,545,246","CDX North American High Yield Index Series 26,...",NaN,06/20/21,nan000,$258000,Buy,NaN
7,NaN,NaN,16,"CDX North American High Yield Index Series 26,...","75,000,000",CDX North American Investment Grade Index Seri...,NaN,06/20/21,$(134000,111000,Buy,NaN
8,NaN,NaN,09,CDX North American Investment Grade Index Seri...,"125,000,000",NaN,(5.000,06/20/14,$(134000,"264,013",Buy,NaN
9,NaN,NaN,09,Barclays Bank PLC,"(3,849,600",NaN,(1.000,06/20/14,$(283000,"290,078",Buy,NaN
